In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', None, '')
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from projects.feed.rank.src import config
from projects.feed.rank.src.model import Model
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
# 载入模型
model_hour = '2020051521'
valid_hour = '2020051523'
model_root = '/search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8'
model_dir = f'{model_root}/{model_hour}'
model_dir = '/home/gezi/tmp/rank/exps/video/v25/dlrm'
print(model_dir)
tfrecord_root = '/search/odin/publicData/CloudS/libowei/rank4/sgsapp/data/video_hour_sgsapp_v1/tfrecords'
tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
info_dir = tfrecord_dir.replace('tfrecords', 'features')
argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
FLAGS(argv)
FLAGS.eager = True
FLAGS.fields_norm = True
config.init()
model = Model()
melt.eager.restore(model, model_dir)

/home/gezi/tmp/rank/exps/video/v25/dlrm


2020-07-02 11:59:04 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-07-02 11:59:04 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-07-02 11:59:04 0:00:00 batch_size: 512 eval_batch_size: 512 batch_size_per_gpu: 512 num_gpus: 0 gpu: [] CUDA_VISIABLE_DEVICES=[-1] work_mode: train distributed: False horovod: False
2020-07-02 11:59:08 0:00:03 input start_hour [2020051520] is different from model last saved hour + 1 as [2020051523]
2020-07-02 11:59:08 0:00:03 start hour before model last train hour will ignore user input start_hour, still use start_hour_from_model
2020-07-02 11:59:08 0:00:03 start_hour 2020051523 end_hour 2020070210
get_num_records:   0%|          | 0/50 [00:00<?, ?it/s]
                                                       
get_num_records:   0%|          | 0/50 [00:00<?, ?it/s]
                                                       

In [6]:
model.layers

In [97]:
melt.print_model(model)

Model: "wide"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
emb/emb (QREmbedding)        multiple                  3000009   
Total params: 3,000,010
Trainable params: 3,000,010
Non-trainable params: 0
_________________________________________________________________
Model: "wide_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
emb/emb (QREmbedding)        multiple                  3000009   
Total params: 3,000,010
Trainable params: 3,000,010
Non-trainable params: 0
_________________________________________________________________
Model: "pooling_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dot_pooling (DotPooling)     multiple                  0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_____________

In [98]:
dataset = Dataset('valid')

In [99]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

799428

In [157]:
batch_size = 1
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [158]:
xs = []
for i, (x, _) in tqdm(enumerate(batches), total=num_steps, ascii=True, desc='loop'):
  xs += [x]
  if i == 1000:
    break
#   if i == num_steps:
#     break

loop:   0%|          | 981/799428 [00:02<26:37, 499.75it/s]  


In [160]:
x = xs[2]
count = 0
for i in range(1000):
  if xs[i]['duration'].numpy() > 0:
    x = xs[i]
    count += 1
    if count == 9:
      break

In [161]:
x['id']

<tf.Tensor: id=460207, shape=(1,), dtype=string, numpy=array([b'3c0b863193036944774\t20504a0K13SJ7O'], dtype=object)>

In [165]:
x['duration']

<tf.Tensor: id=460204, shape=(1, 1), dtype=int64, numpy=array([[81]])>

In [166]:
x['index'].shape

TensorShape([Dimension(1), Dimension(207)])

In [167]:
x['field']

<tf.Tensor: id=460205, shape=(1, 207), dtype=int64, numpy=
array([[ 5260721114193651035, -8919556150788677908, -2869353143095762574,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         1521058157329980945,  1521058157329980945,  1521058157329980945,
         1521058157329980945,  1521058157329980945,  

In [169]:
fields = x['field'].numpy()
fields

array([[ 5260721114193651035, -8919556150788677908, -2869353143095762574,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         1521058157329980945,  1521058157329980945,  1521058157329980945,
         1521058157329980945,  1521058157329980945,  1521058157329980945,
         1521058157329980945,  1521058

In [170]:
fields[0][0] = 123456

In [171]:
fields

array([[              123456, -8919556150788677908, -2869353143095762574,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         1521058157329980945,  1521058157329980945,  1521058157329980945,
         1521058157329980945,  1521058157329980945,  1521058157329980945,
         1521058157329980945,  1521058

In [172]:
x['field'] = tf.constant(fields)

In [173]:
x['field']

<tf.Tensor: id=506557, shape=(1, 207), dtype=int64, numpy=
array([[              123456, -8919556150788677908, -2869353143095762574,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         9117216577038269343,  9117216577038269343,  9117216577038269343,
         1521058157329980945,  1521058157329980945,  1521058157329980945,
         1521058157329980945,  1521058157329980945,  

In [174]:
model.adjust_input(x)

In [175]:
x['field']

<tf.Tensor: id=506561, shape=(1, 207), dtype=int64, numpy=
array([[  0,  18,  85,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,
         53,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,
         53,  53,  53,  53,  53,  53,  53,  54,  54,  54,  54,  54,  54,
         54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,
         54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  55,  55,
         55,  55,  55,  55,  55,  55,  55,  55,  55,  55,  55,  55,  55,
         55,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,
         56,  56,  56,  56,  52,  52,  52,  52,  49,  49,  49,  49,  49,
         49,  49,  49,  49,  49,  49,  49,  49,  49,  50,  50,  50,  50,
         50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,
         50,  50,  50,  50,  50,  50,  50,  51,  51,  51,  51,  51,  51,
         51,  51,  96,  97,  98,  99, 100,  27,  22,  22,  22,  28,  25,
         24,  26,  21,  47,  48,  45,  46,  29,  20,   2,   3,  1

In [176]:
x['visited']

True

In [177]:
float(model(x))

0.06974029541015625

In [156]:
float(model(x))

0.06974029541015625

In [178]:
model.pred

<tf.Tensor: id=506934, shape=(1, 1), dtype=float32, numpy=array([[-2.5907]], dtype=float32)>

In [180]:
model.logit

<tf.Tensor: id=506934, shape=(1, 1), dtype=float32, numpy=array([[-2.5907]], dtype=float32)>

In [181]:
model.prob

<tf.Tensor: id=506925, shape=(1, 1), dtype=float32, numpy=array([[0.0697]], dtype=float32)>

In [182]:
model.prob_click

<tf.Tensor: id=506866, shape=(1, 1), dtype=float32, numpy=array([[0.9495]], dtype=float32)>

In [183]:
model.prob_dur

<tf.Tensor: id=506922, shape=(1, 1), dtype=float32, numpy=array([[0.0245]], dtype=float32)>

In [184]:
model.prob_click ** 1.3 * model.prob_dur ** 0.7

<tf.Tensor: id=506960, shape=(1, 1), dtype=float32, numpy=array([[0.0697]], dtype=float32)>

In [185]:
# indexes = x['index'].numpy()
# print(indexes.shape)
# indexes[0] = indexes[0][1:]
# x['index'] = indexes
# x['index']
x['index'] = x['index'][:,1:]


In [186]:
x['index']

<tf.Tensor: id=506965, shape=(1, 206), dtype=int64, numpy=
array([[ -681005663538039750, -1056610930512986964,  -908709413958721136,
        -6071670189937409566,  6949555566129301704, -4904879907473212121,
         2901991421633787865,  5460007954892354917,  8196366725221085069,
         1324947791478287122, -1875450093904159056,  2548539531688050476,
         8363975984982225395,  3968562212652774878, -7626349362752034242,
        -8077881214077659733,  4648207246762078655,  7226928808955094411,
          755098908229656246, -3462141727220458618, -5717194380768265589,
         4110309556392439779,  6074399845794007325,  4355437864586460014,
         5020648516627835720,  7955363493440204990, -5476655164218273911,
        -3935653589609646529, -5460371247509131008,  5561509050528709250,
         6680411834223800745,  -354537144254491982, -8908814010638033856,
         3220060811559622064, -8797143434110016408,  4375689316409728813,
         5160774465989171078, -8217064762503836421,  

In [187]:
x['field'] = x['field'][:,1:]

In [188]:
x['field']

<tf.Tensor: id=506970, shape=(1, 206), dtype=int64, numpy=
array([[ 18,  85,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,
         53,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,  53,
         53,  53,  53,  53,  53,  53,  54,  54,  54,  54,  54,  54,  54,
         54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  54,
         54,  54,  54,  54,  54,  54,  54,  54,  54,  54,  55,  55,  55,
         55,  55,  55,  55,  55,  55,  55,  55,  55,  55,  55,  55,  55,
         56,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,  56,
         56,  56,  56,  52,  52,  52,  52,  49,  49,  49,  49,  49,  49,
         49,  49,  49,  49,  49,  49,  49,  49,  50,  50,  50,  50,  50,
         50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,
         50,  50,  50,  50,  50,  50,  51,  51,  51,  51,  51,  51,  51,
         51,  96,  97,  98,  99, 100,  27,  22,  22,  22,  28,  25,  24,
         26,  21,  47,  48,  45,  46,  29,  20,   2,   3,  11,  7

In [189]:
x['value'] = x['value'][:,1:]

In [191]:
float(model(x))

0.06974029541015625

In [ ]:
model.deep.before_mlp

In [ ]:
dim = tf.cast(tf.cast(tf.shape(model.deep.before_mlp)[1], tf.float32) ** 0.5, tf.int32)

In [ ]:
intersection = tf.reshape(model.deep.before_mlp, [dim, dim]).numpy()

In [ ]:
# for i in range(dim):
#   intersection[i][i] = 0.

In [ ]:
intersection

In [ ]:
# intersection = tf.minimum(intersection, 5)

In [ ]:
tf.reduce_sum(tf.cast(intersection > 2, tf.int32))

In [ ]:
def rename(field):
  for x in maps:
#     print(x, len(x))
#     print('0', field, x[0], x[1])
    field = field.replace(x[0], x[1])
#     print('1', field, x[0], x[1])
  return field

In [ ]:
other_fields = [x.name.split('/')[-2] for x in model.variables if x.name.endswith('embeddings:0')and not '/emb/emb/' in x.name]
other_fields[model.deep.history_index:model.deep.history_index] = model.deep.history_fields
onehot_fields = gezi.get_global('fields')
maps = [x.strip().split() for x in open('../conf/fields_map.txt')]
onehot_fields2 = [rename(x) for x in onehot_fields]
fields = other_fields + onehot_fields2
[(x, y) for x, y in zip(onehot_fields, onehot_fields2)]

In [ ]:
X = list(range(dim))
X = fields
heat = go.Heatmap(z=intersection,
                  x=X,
                  y=X,
                  xgap=1, ygap=1,
                  )
layout = go.Layout(
width=2000, height=2000,
xaxis_showgrid=False,
yaxis_showgrid=False,
yaxis_autorange='reversed'
)
fig=go.Figure(data=[heat], layout=layout)
py.iplot(fig)

In [ ]:
d = pd.DataFrame(intersection)
d

In [ ]:
x['history'].shape

In [ ]:
l = [(d[i] ** 2).mean() for i in range(len(d))]
l2 = [d[i].mean() for i in range(len(d))]

In [ ]:
[(i, fields[i]) for i in range(len(d)) if l[i] == 0]

In [ ]:
sorted([(l[i], fields[i]) for i in range(len(d))], reverse=True)

In [ ]:
sorted([(l2[i], fields[i]) for i in range(len(d))], reverse=True)

In [ ]:
m = []
m2 = []
for i in range(len(d)):
  for j in range(len(d)):
    if j > i:
      m.append((d[i][j], fields[i], fields[j]))
      m2.append((d[i][j] ** 2, d[i][j], fields[i], fields[j]))

In [ ]:
sorted(m, reverse=True)[:100]

In [ ]:
sorted(m, reverse=False)[:100]

In [ ]:
sorted(m2, reverse=True)[:100]

In [ ]:
x['history'].shape

In [ ]:
index = [u.numpy().decode('gbk').split('\a', 1) for u in info[x['id'][0].numpy()]['index'] if u.numpy().decode('gbk') != '\x01']
[(rename(field), val) for field, val in index]

In [ ]:
len(x['value'][0])

In [ ]:
len([(rename(field), val) for field, val in index if not 'last' in field and not 'long' in field])

In [ ]:
x['mid']

In [ ]:
len([(rename(field), val) for field, val in index if 'last' in field])

In [ ]:
len([(rename(field), val) for field, val in index if 'long' in field])

In [ ]:
from projects.feed.rank.utils.doc_info.KV import KV
kv = KV()
titles = kv.get_titles([u.decode() for u in info[x['id'][0].numpy()]['history'].numpy() if u.decode() not in ['\x01', 'null']])
titles

In [ ]:
kv.get_titles([x['docid'][0].numpy().decode()])

In [ ]:
docs = [u.decode() for u in info[x['id'][0].numpy()]['history'].numpy() if u.decode() != '\x01']
docs

In [ ]:
keys = list(info.keys())
index = [u.numpy().decode('gbk').split('\a', 1) for u in info[keys[0]]['index'] if u.numpy().decode('gbk') != '\x01']
[(rename(field), val) for field, val in index]

In [ ]:
info[keys[0]]

In [ ]:
doc_embs = model.deep.doc_emb(tf.constant([gezi.hash_int64(u) for u in docs]))

In [ ]:
doc2doc = tf.matmul(doc_embs, tf.transpose(doc_embs))

In [ ]:
X = [titles.get(u, 'null') for u in docs]
heat = go.Heatmap(z=doc2doc,
                  x=X,
                  y=X,
                  xgap=1, ygap=1,
                  )
layout = go.Layout(
width=1000, height=1000,
xaxis_showgrid=False,
yaxis_showgrid=False,
yaxis_autorange='reversed'
)
fig=go.Figure(data=[heat], layout=layout)
py.iplot(fig)